In [12]:
import orjson
pwd = '~/projects/gun-overlays/bible/'

In [13]:
with open(pwd+'/roh.json') as f:
    data = orjson.loads(f.read())

for book in data['books']:
    with open(pwd+'/roh/'+book['abbreviation']+'.json', 'wb') as f:
        f.write(orjson.dumps(book.pop('chapters'), option=orjson.OPT_INDENT_2))

with open(pwd+'/roh/index.json', 'wb') as f:
    f.write(orjson.dumps(data, option=orjson.OPT_INDENT_2))

In [15]:
with open(pwd+'/seb.json') as f:
    data = orjson.loads(f.read())

for book in data['books']:
    with open(pwd+'/seb/'+book['abbreviation']+'.json', 'wb') as f:
        f.write(orjson.dumps(book.pop('chapters'), option=orjson.OPT_INDENT_2))

with open(pwd+'/seb/index.json', 'wb') as f:
    f.write(orjson.dumps(data, option=orjson.OPT_INDENT_2))

# Scrape biblia.sk

In [1]:
from time import time, sleep
import binascii
import json
import requests

res = requests.get('https://api2.biblia.sk/api/preklady')
preklady = res.json()['data']
with open('preklady.json', 'w') as f:
    json.dump(preklady, f, indent=2, ensure_ascii=False)

for bibleIndex in [3, 2, 0]: # sep, roh, seb
    preklad = preklady[bibleIndex]
    identifier = preklad['identifier']
    books = preklad['books']

    params = {
        'timestamp': int(time()*1000),
        'key': '597133743677397A24432646294A404E635166546A576E5A7234753778214125',
    }
    for i, book in enumerate(books):
        chapters = {}
        abbr = book['abbreviation']
        for chapter in book['chapters']:
            params['timestamp'] = int(time()*1000)
            addr = f"{identifier}|{abbr}|{chapter}"
            code = binascii.b2a_base64(addr.encode()).strip().decode()
            while True:
                res = requests.get(f"https://api2.biblia.sk/api/get-texts/{code}", params=params)
                print(res.status_code, res.url[:50], addr)
                if res.status_code == 429:
                    print('Too Many Requests, sleeping')
                    sleep(40)
                    continue
                break
            try:
                text = res.json()
            except Exception as e:
                print(res.content)
                raise e
            chapters[str(chapter)] = [v['search_text'].strip() for v in text['chapters'][str(chapter)]]
        book['chapters'] = chapters
        # Write book.json
        with open(f"{identifier}/{abbr}.json", "w") as f:
            json.dump(chapters, f, indent=2, ensure_ascii=False)
    break


200 https://api2.biblia.sk/api/get-texts/c2VwfGdufDE=? sep|gn|1
200 https://api2.biblia.sk/api/get-texts/c2VwfGdufDI=? sep|gn|2
200 https://api2.biblia.sk/api/get-texts/c2VwfGdufDM=? sep|gn|3
200 https://api2.biblia.sk/api/get-texts/c2VwfGdufDQ=? sep|gn|4
200 https://api2.biblia.sk/api/get-texts/c2VwfGdufDU=? sep|gn|5
200 https://api2.biblia.sk/api/get-texts/c2VwfGdufDY=? sep|gn|6
200 https://api2.biblia.sk/api/get-texts/c2VwfGdufDc=? sep|gn|7
200 https://api2.biblia.sk/api/get-texts/c2VwfGdufDg=? sep|gn|8
200 https://api2.biblia.sk/api/get-texts/c2VwfGdufDk=? sep|gn|9
200 https://api2.biblia.sk/api/get-texts/c2VwfGdufDEw? sep|gn|10
200 https://api2.biblia.sk/api/get-texts/c2VwfGdufDEx? sep|gn|11
200 https://api2.biblia.sk/api/get-texts/c2VwfGdufDEy? sep|gn|12
200 https://api2.biblia.sk/api/get-texts/c2VwfGdufDEz? sep|gn|13
200 https://api2.biblia.sk/api/get-texts/c2VwfGdufDE0? sep|gn|14
200 https://api2.biblia.sk/api/get-texts/c2VwfGdufDE1? sep|gn|15
200 https://api2.biblia.sk/api/get

In [5]:
import json
with open('../../src/routes/bible/seb.json.js') as f:
    seb = json.load(f)
sebaliases = {b["abbreviation"]: b["aliases"] for b in seb["books"] if "aliases" in b}
with open('preklady.json') as f:
    preklady = json.load(f)
preklad = preklady[3] # SEP
for book in preklad["books"]:
    book.pop("chapters")
    try:
        book["aliases"] = sebaliases[book["abbreviation"]]
    except KeyError:
        pass
with open('../../src/routes/bible/sep.json.js', 'w') as f:
    json.dump(preklad, f, indent=2, ensure_ascii=False)
